# LangGraph Research Assistant Agent

> **Created by [Build Fast with AI](https://www.buildfastwithai.com)**

This notebook demonstrates how to build a research assistant agent using LangGraph and Gemini 3 Pro.

## What you'll learn:
- Introduction to LangGraph
- Creating stateful agents
- Building tool-using agents
- Implementing agent reasoning loops
- Error handling in agent workflows

In [ ]:
!pip install -q langgraph langchain langchain-google-genai

In [ ]:
import os
from typing import TypedDict, Annotated, List
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor, ToolInvocation
from IPython.display import Image, display, Markdown
import json

In [ ]:
# Configure API key
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY', 'your-api-key-here')

os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

## 1. Define Research Tools

First, let's create tools that our agent can use.

In [ ]:
@tool
def search_arxiv(query: str) -> str:
    """Search for academic papers on ArXiv.
    
    Args:
        query: The search query for papers
    
    Returns:
        String containing paper information
    """
    # Simulated search results
    papers = [
        {
            "title": "Attention Is All You Need",
            "authors": "Vaswani et al.",
            "year": 2017,
            "summary": "Introduced the Transformer architecture for neural networks."
        },
        {
            "title": "BERT: Pre-training of Deep Bidirectional Transformers",
            "authors": "Devlin et al.",
            "year": 2018,
            "summary": "Introduced BERT, a bidirectional transformer for NLP."
        }
    ]
    
    results = f"Found {len(papers)} papers for '{query}':\n\n"
    for i, paper in enumerate(papers, 1):
        results += f"{i}. {paper['title']} ({paper['year']})\n"
        results += f"   Authors: {paper['authors']}\n"
        results += f"   Summary: {paper['summary']}\n\n"
    
    return results

@tool
def search_web(query: str) -> str:
    """Search the web for information.
    
    Args:
        query: The search query
    
    Returns:
        String containing search results
    """
    # Simulated web search
    return f"""Web search results for '{query}':
    
1. Wikipedia: Comprehensive article about {query}
2. Official Documentation: Technical details and API reference
3. Tutorial: Step-by-step guide for beginners
4. Research Blog: Latest developments and insights
"""

@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression.
    
    Args:
        expression: Mathematical expression to evaluate
    
    Returns:
        The result of the calculation
    """
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def summarize_text(text: str) -> str:
    """Summarize a long text.
    
    Args:
        text: The text to summarize
    
    Returns:
        A summary of the text
    """
    # Simple summarization (in production, use LLM)
    words = text.split()
    if len(words) > 50:
        summary = ' '.join(words[:50]) + "..."
    else:
        summary = text
    return f"Summary: {summary}"

# List of all tools
tools = [search_arxiv, search_web, calculate, summarize_text]

print(f"Created {len(tools)} tools for the agent")

## 2. Define Agent State

In [ ]:
class AgentState(TypedDict):
    """State of the research agent."""
    messages: List[dict]
    next_action: str
    intermediate_steps: List[dict]
    final_answer: str

print("Agent state defined")

## 3. Initialize LLM and Tools

In [ ]:
# Initialize Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-3-pro",
    temperature=0.7,
    google_api_key=GOOGLE_API_KEY
)

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

print("LLM initialized with tools")

## 4. Define Agent Nodes

In [ ]:
def agent_node(state: AgentState) -> AgentState:
    """The agent decides what to do next."""
    messages = state['messages']
    
    # Get the last message
    last_message = messages[-1] if messages else {"role": "user", "content": ""}
    
    # Call LLM
    response = llm_with_tools.invoke([
        {"role": msg["role"], "content": msg["content"]}
        for msg in messages
    ])
    
    # Update state
    state['messages'].append({
        "role": "assistant",
        "content": response.content
    })
    
    # Check if tool calls are needed
    if hasattr(response, 'tool_calls') and response.tool_calls:
        state['next_action'] = 'tools'
        state['intermediate_steps'].append({
            "tool_calls": response.tool_calls
        })
    else:
        state['next_action'] = 'end'
        state['final_answer'] = response.content
    
    return state

def tool_node(state: AgentState) -> AgentState:
    """Execute tools."""
    last_step = state['intermediate_steps'][-1]
    tool_calls = last_step.get('tool_calls', [])
    
    # Execute each tool
    tool_results = []
    for tool_call in tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        
        # Find and execute tool
        for tool in tools:
            if tool.name == tool_name:
                result = tool.invoke(tool_args)
                tool_results.append({
                    "tool": tool_name,
                    "result": result
                })
    
    # Add results to messages
    results_text = "\n\n".join([
        f"Tool: {r['tool']}\nResult: {r['result']}"
        for r in tool_results
    ])
    
    state['messages'].append({
        "role": "system",
        "content": f"Tool results:\n{results_text}"
    })
    
    state['next_action'] = 'agent'
    
    return state

def should_continue(state: AgentState) -> str:
    """Determine next step."""
    return state.get('next_action', 'end')

print("Agent nodes defined")

## 5. Build the Agent Graph

In [ ]:
# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

# Set entry point
workflow.set_entry_point("agent")

# Add conditional edges
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        "end": END
    }
)

# Add edge from tools back to agent
workflow.add_edge("tools", "agent")

# Compile the graph
app = workflow.compile()

print("Agent graph built successfully!")

## 6. Run the Research Agent

In [ ]:
def run_research_agent(query: str):
    """Run the research agent with a query."""
    # Initialize state
    initial_state = {
        "messages": [
            {
                "role": "user",
                "content": query
            }
        ],
        "next_action": "agent",
        "intermediate_steps": [],
        "final_answer": ""
    }
    
    print(f"\n{'='*80}")
    print(f"Research Query: {query}")
    print(f"{'='*80}\n")
    
    # Run the agent
    result = app.invoke(initial_state)
    
    # Display results
    print("\nAgent Thought Process:")
    for i, msg in enumerate(result['messages'], 1):
        role = msg['role'].upper()
        content = msg['content']
        print(f"\n[{role}]:")
        print(content[:300] + "..." if len(content) > 300 else content)
    
    print("\n" + "="*80)
    print("\nFinal Answer:")
    display(Markdown(result.get('final_answer', 'No final answer generated')))
    
    return result

# Test the agent
result = run_research_agent(
    "What are the key innovations in transformer architecture? Search for papers and summarize."
)

## 7. Advanced Agent with Memory

In [ ]:
class ResearchAssistant:
    """A stateful research assistant."""
    
    def __init__(self):
        self.conversation_history = []
        self.research_notes = []
    
    def research(self, query: str, save_notes: bool = True):
        """Perform research on a query."""
        # Add to conversation history
        self.conversation_history.append({
            "role": "user",
            "content": query
        })
        
        # Initialize state with history
        initial_state = {
            "messages": self.conversation_history.copy(),
            "next_action": "agent",
            "intermediate_steps": [],
            "final_answer": ""
        }
        
        # Run agent
        result = app.invoke(initial_state)
        
        # Update history
        self.conversation_history = result['messages']
        
        # Save research notes
        if save_notes and result.get('final_answer'):
            self.research_notes.append({
                "query": query,
                "answer": result['final_answer'],
                "tools_used": len(result['intermediate_steps'])
            })
        
        return result['final_answer']
    
    def get_notes(self):
        """Get all research notes."""
        return self.research_notes
    
    def clear_history(self):
        """Clear conversation history."""
        self.conversation_history = []
        print("History cleared")

# Create assistant
assistant = ResearchAssistant()

# Test with multiple queries
queries = [
    "Search for information about transformers in AI",
    "What papers did you find?",
    "Can you summarize the key points?"
]

for query in queries:
    print(f"\n{'='*80}")
    print(f"Query: {query}")
    print(f"{'='*80}\n")
    
    answer = assistant.research(query)
    display(Markdown(answer))

## 8. View Research Notes

In [ ]:
print("Research Notes Summary:\n")
notes = assistant.get_notes()

for i, note in enumerate(notes, 1):
    print(f"{i}. Query: {note['query']}")
    print(f"   Tools used: {note['tools_used']}")
    print(f"   Answer preview: {note['answer'][:100]}...\n")

## 9. Error Handling in Agents

In [ ]:
def safe_agent_node(state: AgentState) -> AgentState:
    """Agent node with error handling."""
    try:
        return agent_node(state)
    except Exception as e:
        print(f"Error in agent: {str(e)}")
        state['next_action'] = 'end'
        state['final_answer'] = f"Error occurred: {str(e)}"
        return state

def safe_tool_node(state: AgentState) -> AgentState:
    """Tool node with error handling."""
    try:
        return tool_node(state)
    except Exception as e:
        print(f"Error in tool execution: {str(e)}")
        state['messages'].append({
            "role": "system",
            "content": f"Tool execution failed: {str(e)}"
        })
        state['next_action'] = 'agent'
        return state

print("Safe agent nodes defined")

## 10. Best Practices for LangGraph Agents

### Key Considerations:

1. **State Management**: Keep state minimal and well-structured
2. **Tool Design**: Make tools focused and reusable
3. **Error Handling**: Always handle errors gracefully
4. **Logging**: Log agent decisions for debugging
5. **Testing**: Test each node independently
6. **Limits**: Set maximum iterations to prevent infinite loops

## Next Steps

- Build multi-agent systems with CrewAI
- Create autonomous agents for complex tasks
- Deploy agents in production with Streamlit

---

## Learn More

Master agent development with the **[Gen AI Crash Course](https://www.buildfastwithai.com/genai-course)** by Build Fast with AI!

**Created by [Build Fast with AI](https://www.buildfastwithai.com)**